In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [2]:
def trans(df):
    df = df.drop(index=[0,1,2])
    df = df.reset_index(drop=True)
    df = df.transpose()
    df.reset_index(drop=True, inplace=True)
    df = df.drop(index=[0])
    df = df.transpose()
    df.columns = df.iloc[0]
    df = df.drop(index=[0])
    df = df.set_index('年月')
    return df

In [3]:
excel_file = 'weeks.xlsx'
dfs = pd.read_excel(excel_file, sheet_name=None)

In [4]:
# Apply the trans function to each sheet and collect the transformed DataFrames
transformed_dfs = [trans(df) for df in dfs.values()]

# Concatenate all transformed DataFrames into a single DataFrame
df = pd.concat(transformed_dfs, ignore_index=True, axis=1)
df = df.dropna(axis=1)

df

c:\Users\joshchiang\AppData\Local\anaconda3\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
c:\Users\joshchiang\AppData\Local\anaconda3\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
c:\Users\joshchiang\AppData\Local\anaconda3\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` 

,7,8,9,10,11,12,13,14,15,24
年月,,,,,,,,,,
2000-09-16,29570,19741.8,9828.2,8840.2,12354693.151,1951000,2004000,-53000,7.56,-4.27
2000-09-22,56040,111303.894,-55263.894,-62307.579,12214510.778,3460000,8748000,-5288000,7.3,-6.26
2000-09-30,46911,8984.528,37926.472,38041.032,13012654.388,2539000,1022000,1517000,8.01,-6.46
2000-10-07,19940,31036.908,-11096.908,-9183.383,12987237.997,1493000,1710000,-217000,8.05,2.7
2000-10-13,67871,133416.781,-65545.781,-64883.932,12862412.249,4382000,7875000,-3493000,7.89,-7.49
...,...,...,...,...,...,...,...,...,...,...
2024-11-08,1027410.577,1204821.951,-177411.374,-167858.135,169707961.066,131310504,131322776,-12272,45.15,3.4
2024-11-15,1607186.143,1094130.952,513055.191,532481.78,169381375.667,152650753,147522590,5128163,44.71,-3.44
2024-11-22,1174824.405,1133793.597,41030.808,22469.111,169425554.646,124450090,116409181,8040909,44.6,0.71


In [ ]:
df[df.columns[-1]] = df[df.columns[-1]].shift(periods = 1)
df.rename(columns={df.columns[-1]: 'shift_ret'}, inplace=True)
df

In [ ]:
df = df.dropna(axis=0)
df['direction'] = np.where(df['shift_ret'] <= -1.5, 1,
                           np.where((df['shift_ret'] > -1.5) & (df['shift_ret'] < 0), -1, 0))
df

In [ ]:
# Calculate quantile ranks for each column except 'shift_ret', 'direction'
quantile_columns = df.columns.difference(['shift_ret', 'direction'])
for col in quantile_columns:
    df[str(col) + '_QuantileRank'] = pd.qcut(df[col].fillna(df[col].median()), 5, labels=False, duplicates='drop') + 1
df

In [ ]:
dfs = df[df['direction'] != -1]
dfs

In [ ]:
# Create the table for each column with specified metrics
tables = {}
for col in quantile_columns:
    avg_shift_ret_table = df.groupby(str(col) + '_QuantileRank').agg(
        Avg_shift_ret=('shift_ret', 'mean')
    )
    
    other_metrics_table = dfs.groupby(str(col) + '_QuantileRank').agg(
        Hit_Rate=('shift_ret', lambda x: (x > 0).mean()),
        Good_Rate=('direction', lambda x: (x == 1).mean() / ((x == 1).mean() + (x == 0).mean()))
    )
    
    table = avg_shift_ret_table.join(other_metrics_table)
    
    # table['WOE'] = np.log(table['Good_Rate'] / (1 - table['Good_Rate']))
    # table['IV'] = (table['Good_Rate'] - (1 - table['Good_Rate'])) * table['WOE']
    tables[col] = table

for col, table in tables.items():
    print(f"Table for column {col}")
    print(table)

In [ ]:
tables[0]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Assuming 'tables' is a dictionary of DataFrames
# Example: tables = {0: df1, 1: df2, ..., 200: df201}

# Create a PDF file to save all plots
file_path = 'U:/【全權委託組】/intern/加權指數跌損預警模型/123.pdf'
with PdfPages(file_path) as pdf:
    for i, table in tables.items():
        plots = pd.DataFrame({
            'Avg_shift_ret': table['Avg_shift_ret'],
            'Hit_Rate': table['Hit_Rate'],
            'Good_Rate': table['Good_Rate']
        }, index=table.index)
        
        # Plotting
        ax = plots.plot(grid=True, figsize=(12, 6))
        ax.set_title(f'Table {i}')
        
        # Save the current plot to the PDF
        pdf.savefig(ax.figure)
        plt.close(ax.figure)


In [ ]:
plots = pd.DataFrame({'Avg_shift_ret': tables[200]['Avg_shift_ret'], 'Hit_Rate': tables[200]['Hit_Rate'], 'Good_Rate': tables[200]['Good_Rate']}, index=tables[200].index)

plots.plot(grid=True, figsize=(12,6))

In [ ]:
#Fit LogisticRegression
from sklearn.linear_model import LogisticRegression

#Accuracy Metrics

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


#Confusion Matrix
from sklearn.metrics import confusion_matrix